# 加载模块

In [32]:
import sys
sys.path.append('/score_card_model/score_card_model/')

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from IPython.core.interactiveshell import InteractiveShell
from tqdm import tqdm
InteractiveShell.ast_node_interactivity = 'all'
warnings.filterwarnings('ignore')


# 数据准备

In [34]:
df_train = pd.read_excel('./LoanStats_2018Q2.xlsx')

In [35]:
df_train.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,disbursement_method,debt_settlement_flag
0,NaN,10000.0,10000.0,10000.0,36 months,20.39%,373.63,D,D4,Realtor,...,5.0,99.2,1.0,4.0,0.0,0.0,NaN,N,Cash,N
1,NaN,8000.0,8000.0,8000.0,36 months,6.83%,246.40,A,A3,City Attorney,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Cash,N
2,NaN,20000.0,20000.0,20000.0,60 months,6.83%,394.43,A,A3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,DirectPay,N
3,NaN,16000.0,16000.0,16000.0,36 months,14.03%,547.08,C,C2,SA Counselor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Cash,N
4,NaN,1000.0,1000.0,1000.0,36 months,23.87%,39.17,E,E2,Senior Electrical Engineer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Cash,N


## 建立统计dataframe

In [ ]:
df_summary = dataSetSummary(df_train)
df_summary


## 剔除缺失率高（大于90%）的特征

In [ ]:
df_train = dataSetElimNullFeat(df_train, df_summary)

In [ ]:
df_summary = dataSetSummary(df_train)
df_summary

## 剔除常值特征（特征某个取值占比超过95%）

In [ ]:
df_train = dataSetElimInvarFeat(df_train, df_summary)

In [ ]:
df_summary = dataSetSummary(df_train)
df_summary

## 删除贷后特征 & 删除空行和重复行 & 60期的行

In [ ]:
list_featureLoanAfter = ['sub_grade', 'grade', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
                         'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
                         'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
                         'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
                         'collections_12_mths_ex_med', 'policy_code', 'disbursement_method',
                         'id', 'desc', 'emp_title', 'title']
for feature in list_featureLoanAfter:
    if feature in list(df_train.columns):
        df_train.drop(feature, axis=1, inplace=True)
df_train = df_train.drop_duplicates().dropna(how='all', axis=0)
df_summary = dataSetSummary(df_train)
df_summary

## 转化特征

In [ ]:
# term, 期数
df_train['term'] = df_train['term'].apply(lambda x: pd.to_numeric(x.replace('months', ''), errors='coerce'))
df_train = df_train.loc[df_train['term'] == 36]
df_train = df_train.drop('term', axis=1)

In [ ]:
# emp_length, 工作年限
dict_emp_length = {'< 1 year': '00', '1 year': '01', '2 years': '02',
                   '3 years': '03', '4 years': '04', '5 years': '05',
                   '6 years': '06', '7 years': '07', '8 years': '08',
                   '9 years': '09', '10+ years': '10'}
df_train['emp_length'] = df_train['emp_length'].map(dict_emp_length)

In [ ]:
# delinq_2yrs, 过去两年逾期次数, 存在一个错误数据
df_train['delinq_2yrs'] = df_train['delinq_2yrs'].apply(pd.to_numeric, errors='coerce')

In [ ]:
# total_acc, 借款人当前信用额度总个数, 存在一个错误数据
df_train['total_acc'] = df_train['total_acc'].apply(pd.to_numeric, errors='coerce')

## 创建新特征

In [ ]:
# gap_months, 最早开户日至申请日之间的月份
df_train['earliest_to_issue'] = df_train.apply(lambda x: get_monthsbetween(x['earliest_cr_line'], x['issue_d']), axis=1)
df_train = df_train.drop(['earliest_cr_line', 'issue_d'], axis=1)



## 创建目标变量（y值）


In [ ]:
df_train['y'] = df_train['loan_status'].apply(lambda x: 0 if 'Fully Paid' in x else 1)
df_train = df_train.drop('loan_status', axis=1)

In [ ]:
df_summary = dataSetSummary(df_train)
df_summary

# EDA

## 整理连续型特征、类别型特征

In [ ]:
cols_discrete = ['emp_length', 'home_ownership', 'verification_status', 'purpose', 'addr_state',
                 'zip_code']
cols_continuous = []
for feat in list(df_summary['featureName']):
    if feat not in cols_discrete:
        cols_continuous.append(feat)

# 类别性
cols_discrete
# 连续型
cols_continuous

## 类别型特征

In [ ]:
# discrete features
plot_discrete(df_train, cols_discrete, 'y')

## 连续型特征

In [ ]:
# continuous features
plot_continuous(df_train, cols_continuous, 'y')

# 卡方分箱

## 分箱前的异常值&缺失值处理

In [ ]:
# emp_length, 类别型,不满空箱5%的要求,用众数填充
df_train['emp_length'] = df_train['emp_length'].fillna(df_train['emp_length'].mode().values[0])

# annual_inc, 连续型,不满空箱5%的要求,用中位数填充
df_train['annual_inc'] = df_train['annual_inc'].fillna(df_train['annual_inc'].median())

# delinq_2yrs, 连续型,不满空箱5%的要求,用中位数填充
df_train['delinq_2yrs'] = df_train['delinq_2yrs'].fillna(df_train['delinq_2yrs'].median())

# inq_last_6mths, 连续型,不满空箱5%的要求,用中位数填充
df_train['inq_last_6mths'] = df_train['inq_last_6mths'].fillna(df_train['inq_last_6mths'].median())

# mths_since_last_delinq, 连续型, 业务可解释,用0填充
df_train['mths_since_last_delinq'] = df_train['mths_since_last_delinq'].fillna(0)

# open_acc, 连续型,不满空箱5%的要求,用中位数填充
df_train['open_acc'] = df_train['open_acc'].fillna(df_train['open_acc'].median())

# pub_rec, 连续型,不满空箱5%的要求,用中位数填充
df_train['pub_rec'] = df_train['pub_rec'].fillna(df_train['pub_rec'].median())

# revol_util, 连续型,不满空箱5%的要求,用中位数填充
df_train['revol_util'] = df_train['revol_util'].fillna(df_train['revol_util'].median())

# total_acc, 连续型,不满空箱5%的要求,用中位数填充
df_train['total_acc'] = df_train['total_acc'].fillna(df_train['total_acc'].median())

# pub_rec_bankruptcies, 连续型,大致满足空箱5%的要求,用-1填充
df_train['pub_rec_bankruptcies'] = df_train['pub_rec_bankruptcies'].fillna(-1)

# earliest_to_issue, 不满空箱5%的要求,用中位数填充
df_train['earliest_to_issue'] = df_train['earliest_to_issue'].fillna(df_train['earliest_to_issue'].median())

In [ ]:
df_summary = dataSetSummary(df_train)
df_summary

## 卡方分箱参数

In [ ]:
# 分割训练集，测试集, 注意这里是为了分箱的预处理,所以不能去掉y值
X_train, X_test = train_test_split(df_train,
                                   test_size=0.2, random_state=1,
                                   stratify=df_train['y'])
# 最大分箱数目
max_intervals = 8
# 单项数据量最小占比
min_pnt = 0.05

### 离散型特征

In [ ]:
# 将离散型特征分为三类
# 1: 离散有序特征 feat_order
# 2: 离散无序分箱数小于等于阈值 feat_discrete_less
# 3: 离散无序分箱数大于阈值 feat_discrete_more
feat_discrete_order = ['emp_length']
feat_discrete_less = []
feat_discrete_more = []
for feat in cols_discrete:
    if feat in feat_discrete_order:
        continue
    elif len(set(X_train[feat])) <= max_intervals:
        feat_discrete_less.append(feat)
    else:
        feat_discrete_more.append(feat)

# 展开看看
feat_discrete_order
feat_discrete_less
feat_discrete_more

In [ ]:
discrete_order = {'emp_length':{'00': 0, '01': 1, '02': 2, '03': 3, '04': 4,
                                '05': 5, '06': 6, '07': 7, '08': 8, '09': 9, 
                                '10': 10}}
dict_discrete_featToBins, dict_discrete_iv, dict_discrete_woe = chi2_cutting_discrete(X=X_train, 
                                                                                      feat_list=feat_discrete_less + feat_discrete_order, 
                                                                                      y='y', 
                                                                                      special_feat_val={}, 
                                                                                      max_intervals=max_intervals, 
                                                                                      min_pnt=min_pnt,
                                                                                      discrete_order=discrete_order, 
                                                                                      mono_expect={'emp_length':{'shape':'mono', 'u':False}}, 
                                                                                      print_process=True, 
                                                                                      pr_process_time=True)

### 连续型特征

In [ ]:
# 无序离散多
feat_discrete_more
# 连续型
cols_continuous
# 连续型全部需要检查单调性
mono_contin = {i:{'shape': 'mono', 'u': False} for i in cols_continuous} 
mono_contin

In [ ]:
dict_contin_featToBins, dict_contin_iv, dict_contin_woe = chi2_cutting_continuous(X_train, 
                                                                                  feat_list = feat_discrete_more + cols_continuous, 
                                                                                  y='y',
                                                                                  discrete_more_feats=feat_discrete_more,
                                                                                  special_feat_val={'pub_rec_bankruptcies':[-1]}, 
                                                                                  max_intervals=8, 
                                                                                  min_pnt=0.05,
                                                                                  mono_expect=mono_contin,
                                                                                  print_process=True, pr_process_time=True)

## 整理分组取值、WOE系数、特征IV

In [ ]:
# 分组取值
dict_featToBins = {}
dict_featToBins.update(dict_contin_featToBins)
dict_featToBins.update(dict_discrete_featToBins)

# WOE系数
dict_featWoe = {}
dict_featWoe.update(dict_discrete_woe)
dict_featWoe.update(dict_contin_woe)

# 特征IV
dict_featIv = {}
dict_featIv.update(dict_discrete_iv)
dict_featIv.update(dict_contin_iv)

# 对训练集进行WOE转换

In [ ]:
X_train_woe, dict_featIv_woe = dataset_woe_trans(X=X_train, dict_feativ=dict_featIv,
                                                 feat_disc=cols_discrete, 
                                                 dict_feattobins=dict_featToBins, 
                                                 dict_featwoe=dict_featWoe)

# 基于IV剔除特征

In [ ]:
# 变量IV普遍都比较低, 设置阈值为0.01
# logit_summary(X_train_woe)
iv_thres = 0.01
list_feats_l_iv = feats_iv_proc(dict_feativ=dict_featIv_woe, iv_thres=iv_thres)
list_feats_l_iv
X_train_woe = X_train_woe.drop(list_feats_l_iv, axis=1)

# 基于相关系数(皮尔逊)剔除特征

In [ ]:
# 3.1.1的matplotlib有BUG, 可能显示的热力图会不完全
# 默认阈值为corr=0.7
# logit_summary(X_train_woe)
list_feats_h_corr = feats_corr_proc(X=X_train_woe, dict_feativ=dict_featIv_woe)
list_feats_h_corr
X_train_woe = X_train_woe.drop(list_feats_h_corr, axis=1)

# 基于VIF消除多重共线性

In [ ]:
# 默认vif阈值为10
# logit_summary(X_train_woe)
list_feats_h_vif = feats_vif_proc(X_train_woe, dict_feativ=dict_featIv_woe)
list_feats_h_vif
X_train_woe = X_train_woe.drop(list_feats_h_vif, axis=1)

# 回归系数的显著性检验

In [ ]:
# 剔除前的的显著性检验
logit_summary(X_train_woe)

In [ ]:
# 显著因子默认为5%
list_feats_h_pval = feats_pvalue_proc(X_woe=X_train_woe, targetname='y')
list_feats_h_pval
X_train_woe = X_train_woe.drop(list_feats_h_pval, axis=1)

In [ ]:
# 检查最终的显著性
logit_summary(X_train_woe)

# 基于sklearn进行逻辑回归建模

## 建模

In [ ]:
# 个人习惯更倾向于用sklearn进行建模
lr = LogisticRegression()
grid_param = {'penalty': ['l1', 'l2'],
              'C': [0.1, 0.3, 0.5, 1]}
lr = GridSearchCV(estimator=lr, param_grid=grid_param)
# 目标变量
targetname = 'y'
# 准备数据
X_woe = X_train_woe.drop(targetname, axis=1)
y_woe = X_train_woe[targetname]
# 拟合
lr.fit(X_woe, y_woe)

## 基于模型进行预测类别、概率、分数

In [ ]:
# 预测
X_train_woe[targetname + '_pred'] = lr.predict(X_woe)
# 预测概率
X_train_woe[targetname + '_proba'] = [i[1] for i in lr.predict_proba(X_woe)]
# 默认基础分base_score=500
# 默认odd翻倍增加分数 pdo=20
X_train_woe['score'] = X_train_woe[targetname + '_proba'].apply(lambda x: proba_to_score(x))

## 梳理入模特征


In [ ]:
list_feats_inmod_woe = list(X_woe.columns)
list_feats_inmod = [feat[:len(feat)-4] for feat in list_feats_inmod_woe]
list_feats_inmod

# score分布与特征字典

## score & badrate分布

In [ ]:
plot_score_badrate(X_train_woe)

## 入模特征的badrate分布

In [ ]:
plot_feat_badrate(X_train, list_feats_inmod, cols_discrete, dict_featToBins)

## 整理特征分数组

In [ ]:
df_feat_group = model_feats_score(X_woe, estimator=lr.best_estimator_,
                                  dict_feattoobins=dict_featToBins, 
                                  dict_featwoe=dict_featWoe)
df_feat_group

## 对测试集进行预测, 整理未入模的特征取值

In [ ]:
X_test_woe, dict_inv_featvals = test_trans_pred(X_test, lr.best_estimator_, 
                                                list_feats_inmod, 
                                                dict_featToBins, dict_featWoe, cols_discrete)
dict_inv_featvals

# 模型效果评估

## 训练集效果

### AUC曲线

In [ ]:
auc_train = model_roc_auc(X_train_woe)

### K-S曲线

In [ ]:
ks_train = distin_ks(X_train_woe)

### Gini

In [ ]:
distin_gini(X_train_woe)

## 测试集效果

### AUC曲线

In [ ]:
auc_test = model_roc_auc(X_test_woe)

### K-S曲线

In [ ]:
ks_test = distin_ks(X_test_woe)

### Gini

In [ ]:
distin_gini(X_test_woe)

## 模型的PSI（Train vs Test）

In [ ]:
psi, df_psi = model_psi(X_train_woe, X_test_woe)
psi
df_psi

## 模型的LIFT曲线

In [ ]:
plot_model_lift(X_train_woe)